In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Parámetros del problema
Lx = 20  # Longitud en la dirección x
Ly = 20  # Longitud en la dirección y
Nx = 20   # Número de nodos en x
Ny = 20   # Número de nodos en y
dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)

# Propiedades del medio poroso
mu = 1.0  # Viscosidad
phi = 0.2  # Porosidad
cT = 1.0  # Constante relacionada con la compresibilidad
rho_agua = 1000.0  # Densidad del agua
k11 = 1.0  # Permeabilidad en dirección x (definida por el usuario)
k22 = 1.0  # Permeabilidad en dirección y (definida por el usuario)
dt = 0.01  # Paso de tiempo
T = 1  # Tiempo total de simulación

# Condiciones iniciales y de frontera
p0 = 508.56  # Presión inicial en cada nodo (excepto el punto de inyección)

# Definir los nodos de inicio y fin
start_node = (1, 1)
end_node = (Nx-1, Ny-1)

# Inicialización de matrices para la presión y la velocidad
p_backward = np.full((Nx, Ny), p0)  # Inicializar con la presión inicial en todos los nodos
p_backward[nodo_inicial] = p0  # Establecer la presión inicial en el punto de inyección

# Fronteras cerradas en tres lados
p_backward[:, 0] = p_backward[:, 1]  # Frontera izquierda
p_backward[:, -1] = p_backward[:, -2]  # Frontera derecha
p_backward[0, :] = p_backward[1, :]  # Frontera inferior

# Realizar la simulación (Euler hacia atrás)
timesteps = int(T / dt)
rho = rho_agua

# Crear posiciones de los nodos
x_nodes = np.linspace(0, Lx, Nx)
y_nodes = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x_nodes, y_nodes)

# Función para visualizar la simulación en un tiempo específico
def visualize_simulation(t):
    current_time = int(t / dt)
    
    # Copiar la matriz de presiones actual para usarla en el cálculo
    p_current = np.copy(p_backward)

    # Ciclo principal
    for _ in range(1, current_time + 1):
        # Calcular las derivadas parciales de la presión
        d2p_dx2 = np.gradient(np.gradient(p_current, dx, axis=0), dx, axis=0)
        d2p_dy2 = np.gradient(np.gradient(p_current, dy, axis=1), dy, axis=1)

        # Construir matriz del sistema para la ecuación de flujo (Euler hacia atrás)
        A = np.eye((end_node[0] - start_node[0] + 1) * (end_node[1] - start_node[1] + 1))  # Matriz identidad
        b = np.zeros((end_node[0] - start_node[0] + 1) * (end_node[1] - start_node[1] + 1))  # Vector de términos independientes

        idx = 0
        for i in range(start_node[0], end_node[0] + 1):
            for j in range(start_node[1], end_node[1] + 1):
                A[idx, idx] = 1 + dt * rho_agua * phi * cT * (k11 / dx**2 + k22 / dy**2)
                if i > start_node[0]:
                    A[idx, idx - (end_node[1] - start_node[1] + 1)] = -dt * rho_agua * phi * cT * k22 / dy**2
                if i < end_node[0]:
                    A[idx, idx + (end_node[1] - start_node[1] + 1)] = -dt * rho_agua * phi * cT * k22 / dy**2
                if j > start_node[1]:
                    A[idx, idx - 1] = -dt * rho_agua * phi * cT * k11 / dx**2
                if j < end_node[1]:
                    A[idx, idx + 1] = -dt * rho_agua * phi * cT * k11 / dx**2

                rhs = k11 * d2p_dx2[i, j] + k22 * d2p_dy2[i, j]
                b[idx] = p_current[i, j] + dt * rho_agua * phi * cT * rhs
                idx += 1

        # Resolver el sistema de ecuaciones lineales Ax = b
        x = np.linalg.solve(A, b)

        # Actualizar la matriz de presiones con los nuevos valores
        idx = 0
        for i in range(start_node[0], end_node[0] + 1):
            for j in range(start_node[1], end_node[1] + 1):
                p_current[i, j] = x[idx]
                idx += 1

    # Visualizar la simulación con nodos, malla y campo de velocidades
    plt.figure(figsize=(8, 8))
    plt.imshow(p_current, cmap='viridis', extent=[0, Lx, 0, Ly], origin='lower')
    plt.colorbar(label='Presión [kPa]')
    plt.scatter(X, Y, c='red', s=5, marker='.')  # Añadir nodos
    plt.title(f'Simulación en el tiempo t = {t:.2f} [s]')
    plt.xlabel('x * 10 [m]')
    plt.ylabel('y * 10 [m]')
    plt.grid(True, linestyle='--', linewidth=0.5, color='white')

    plt.show()

# Crear el deslizador interactivo
interact(visualize_simulation, t=widgets.FloatSlider(min=0, max=T, step=dt, value=0, description='Tiempo (t) [s]'))

interactive(children=(FloatSlider(value=0.0, description='Tiempo (t) [s]', max=1.0, step=0.01), Output()), _do…

<function __main__.visualize_simulation(t)>